# 事前準備


## matplotlib 日本語化


In [105]:
!pip install japanize-matplotlib



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## ライブラリのインポート


In [106]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression as LR
import japanize_matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
# データを表示する時に列を省略しないためのおまじない
pd.options.display.max_columns = None

%matplotlib inline

# #日本語化matplotlibのインポート & 日本語のフォントを指定
plt.rcParams['font.family'] = 'IPAexGothic'


# データの読み込み


In [107]:
train = pd.read_csv("./input/train_all.csv")
test = pd.read_csv("./input/test_all.csv")

# 投稿用のサンプルファイルの読み込み
sample = pd.read_csv("./input/sample_submit.csv", header=None)


# 前処理


## 外れ値の除外


In [108]:
train = train[train["id"] != 15127]  # yの値が大きく外れたデータ
train = train[train["id"] != 15699]  # yの値が0のデータ
train = train[train["id"] != 14071]  # 木の中でyの値が外れたデータ
train = train[train["id"] != 14911]  # 金の中でyの値が外れたデータ
train = train[train["y"] < 50000]  # yの値が50000以下

## yをあらかじめ取り出す

In [109]:
y_train = train["y"]
train = train.drop("y", axis=1)
min_values = y_train.min()

## 結合


In [110]:
# 結合前にtrainというフラグを作り、trainデータは1、testデータは0とする
train["train"] = 1
test["train"] = 0

# 結合
datas = pd.concat([train, test], sort=False)


## 不要なカラムの削除


In [111]:
drop_list = [
    "tv",
    "referee",
    "time",
    "home",
    "away",
    "home_01", "home_02", "home_03", "home_04", "home_05", "home_06", "home_07", "home_08", "home_09", "home_10", "home_11",
    "away_01", "away_02", "away_03", "away_04", "away_05", "away_06", "away_07", "away_08", "away_09", "away_10", "away_11",
    "address",
    "humidity",
    "temperature",
]
datas = datas.drop(drop_list, axis=1)

datas.head(5)


,id,year,stage,match,gameday,stadium,home_score,away_score,weather,home_team,away_team,capa,train
0,13994,2012,Ｊ１,第１節第１日,03/10(土),ユアテックスタジアム仙台,1,0,雨,ベガルタ仙台,鹿島アントラーズ,19694,1
1,13995,2012,Ｊ１,第１節第１日,03/10(土),豊田スタジアム,1,0,屋内,名古屋グランパス,清水エスパルス,40000,1
2,13996,2012,Ｊ１,第１節第１日,03/10(土),万博記念競技場,2,3,晴一時雨,ガンバ大阪,ヴィッセル神戸,21000,1
3,13997,2012,Ｊ１,第１節第１日,03/10(土),エディオンスタジアム広島,1,0,曇一時雨のち晴,サンフレッチェ広島,浦和レッズ,50000,1
4,13998,2012,Ｊ１,第１節第１日,03/10(土),札幌ドーム,0,0,屋内,コンサドーレ札幌,ジュビロ磐田,39232,1


## 日付関係


In [112]:
datas["MONTH"] = datas["gameday"].apply(lambda x: x[0:2])
datas["DAY"] = datas["gameday"].apply(lambda x: x[3:5])
datas["WEEK"] = datas["gameday"].apply(
    lambda x: x[x.find("(")+1:x.find("(")+2])
datas = pd.get_dummies(datas, columns=["WEEK"], drop_first=True)
datas = datas.drop("gameday", axis=1)


## 試合？関係


In [113]:
datas["stage"] = datas["stage"].apply(lambda x: x[1])
datas["match_num"] = datas["match"].apply(
    lambda x: int(re.findall(r'\d+', x)[0]))
datas["match_day"] = datas["match"].apply(
    lambda x: int(re.findall(r'\d+', x)[1]))
datas = datas.drop("match", axis=1)


## チーム関係


In [114]:
datas = pd.get_dummies(datas, columns=["home_team"], drop_first=True)
# datas = pd.get_dummies(datas, columns=["away_team"], drop_first=True)
# datas = datas.drop("home_team", axis=1)
datas = datas.drop("away_team", axis=1)


## スタジアム関係


In [115]:
datas = pd.get_dummies(datas, columns=["stadium"], drop_first=True)
# datas = datas.drop("stadium", axis=1)

## 説明変数および目的変数の抽出


- 学習データ（train）データからは説明変数と目的変数を、評価データ（test）からは説明変数のみを取り出します
- サンプルコードでは次のように変数の名前を付けます<br>
  - 学習データの説明変数： trainX
  - 学習データの目的変数： y
  - 評価データの説明変数： testX


In [116]:
datas.columns


Index(['id', 'year', 'stage', 'home_score', 'away_score', 'weather', 'capa',
       'train', 'MONTH', 'DAY',
       ...
       'away_team_横浜ＦＣ', 'away_team_水戸ホーリーホック', 'away_team_浦和レッズ',
       'away_team_清水エスパルス', 'away_team_湘南ベルマーレ', 'away_team_鹿島アントラーズ',
       'away_team_ＦＣ岐阜', 'away_team_ＦＣ東京', 'away_team_ＦＣ町田ゼルビア',
       'away_team_Ｖ・ファーレン長崎'],
      dtype='object', length=102)

In [117]:
datas.head()


,id,year,stage,home_score,away_score,weather,capa,train,MONTH,DAY,WEEK_日,WEEK_月,WEEK_木,WEEK_水,WEEK_火,WEEK_金,match_num,match_day,home_team_アルビレックス新潟,home_team_カターレ富山,home_team_カマタマーレ讃岐,home_team_ガイナーレ鳥取,home_team_ガンバ大阪,home_team_ギラヴァンツ北九州,home_team_コンサドーレ札幌,home_team_サガン鳥栖,home_team_サンフレッチェ広島,home_team_ザスパクサツ群馬,home_team_ザスパ草津,home_team_ジェフユナイテッド千葉,home_team_ジュビロ磐田,home_team_セレッソ大阪,home_team_ファジアーノ岡山,home_team_ベガルタ仙台,home_team_モンテディオ山形,home_team_ロアッソ熊本,home_team_ヴァンフォーレ甲府,home_team_ヴィッセル神戸,home_team_京都サンガF.C.,home_team_名古屋グランパス,home_team_大分トリニータ,home_team_大宮アルディージャ,home_team_川崎フロンターレ,home_team_徳島ヴォルティス,home_team_愛媛ＦＣ,home_team_東京ヴェルディ,home_team_松本山雅ＦＣ,home_team_柏レイソル,home_team_栃木ＳＣ,home_team_横浜Ｆ・マリノス,home_team_横浜ＦＣ,home_team_水戸ホーリーホック,home_team_浦和レッズ,home_team_清水エスパルス,home_team_湘南ベルマーレ,home_team_鹿島アントラーズ,home_team_ＦＣ岐阜,home_team_ＦＣ東京,home_team_ＦＣ町田ゼルビア,home_team_Ｖ・ファーレン長崎,away_team_アルビレックス新潟,away_team_カターレ富山,away_team_カマタマーレ讃岐,away_team_ガイナーレ鳥取,away_team_ガンバ大阪,away_team_ギラヴァンツ北九州,away_team_コンサドーレ札幌,away_team_サガン鳥栖,away_team_サンフレッチェ広島,away_team_ザスパクサツ群馬,away_team_ザスパ草津,away_team_ジェフユナイテッド千葉,away_team_ジュビロ磐田,away_team_セレッソ大阪,away_team_ファジアーノ岡山,away_team_ベガルタ仙台,away_team_モンテディオ山形,away_team_ロアッソ熊本,away_team_ヴァンフォーレ甲府,away_team_ヴィッセル神戸,away_team_京都サンガF.C.,away_team_名古屋グランパス,away_team_大分トリニータ,away_team_大宮アルディージャ,away_team_川崎フロンターレ,away_team_徳島ヴォルティス,away_team_愛媛ＦＣ,away_team_東京ヴェルディ,away_team_松本山雅ＦＣ,away_team_柏レイソル,away_team_栃木ＳＣ,away_team_横浜Ｆ・マリノス,away_team_横浜ＦＣ,away_team_水戸ホーリーホック,away_team_浦和レッズ,away_team_清水エスパルス,away_team_湘南ベルマーレ,away_team_鹿島アントラーズ,away_team_ＦＣ岐阜,away_team_ＦＣ東京,away_team_ＦＣ町田ゼルビア,away_team_Ｖ・ファーレン長崎
0,13994,2012,１,1,0,雨,19694,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,13995,2012,１,1,0,屋内,40000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2,13996,2012,１,2,3,晴一時雨,21000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,13997,2012,１,1,0,曇一時雨のち晴,50000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,13998,2012,１,0,0,屋内,39232,1,03,10,False,

In [118]:
# 使わない説明変数を設定
drop_list = [
    "weather",
    "year",
    "home_score",
    "away_score",
]


In [119]:
datas = datas.drop(drop_list, axis=1)


In [120]:
datas


,id,stage,capa,train,MONTH,DAY,WEEK_日,WEEK_月,WEEK_木,WEEK_水,WEEK_火,WEEK_金,match_num,match_day,home_team_アルビレックス新潟,home_team_カターレ富山,home_team_カマタマーレ讃岐,home_team_ガイナーレ鳥取,home_team_ガンバ大阪,home_team_ギラヴァンツ北九州,home_team_コンサドーレ札幌,home_team_サガン鳥栖,home_team_サンフレッチェ広島,home_team_ザスパクサツ群馬,home_team_ザスパ草津,home_team_ジェフユナイテッド千葉,home_team_ジュビロ磐田,home_team_セレッソ大阪,home_team_ファジアーノ岡山,home_team_ベガルタ仙台,home_team_モンテディオ山形,home_team_ロアッソ熊本,home_team_ヴァンフォーレ甲府,home_team_ヴィッセル神戸,home_team_京都サンガF.C.,home_team_名古屋グランパス,home_team_大分トリニータ,home_team_大宮アルディージャ,home_team_川崎フロンターレ,home_team_徳島ヴォルティス,home_team_愛媛ＦＣ,home_team_東京ヴェルディ,home_team_松本山雅ＦＣ,home_team_柏レイソル,home_team_栃木ＳＣ,home_team_横浜Ｆ・マリノス,home_team_横浜ＦＣ,home_team_水戸ホーリーホック,home_team_浦和レッズ,home_team_清水エスパルス,home_team_湘南ベルマーレ,home_team_鹿島アントラーズ,home_team_ＦＣ岐阜,home_team_ＦＣ東京,home_team_ＦＣ町田ゼルビア,home_team_Ｖ・ファーレン長崎,away_team_アルビレックス新潟,away_team_カターレ富山,away_team_カマタマーレ讃岐,away_team_ガイナーレ鳥取,away_team_ガンバ大阪,away_team_ギラヴァンツ北九州,away_team_コンサドーレ札幌,away_team_サガン鳥栖,away_team_サンフレッチェ広島,away_team_ザスパクサツ群馬,away_team_ザスパ草津,away_team_ジェフユナイテッド千葉,away_team_ジュビロ磐田,away_team_セレッソ大阪,away_team_ファジアーノ岡山,away_team_ベガルタ仙台,away_team_モンテディオ山形,away_team_ロアッソ熊本,away_team_ヴァンフォーレ甲府,away_team_ヴィッセル神戸,away_team_京都サンガF.C.,away_team_名古屋グランパス,away_team_大分トリニータ,away_team_大宮アルディージャ,away_team_川崎フロンターレ,away_team_徳島ヴォルティス,away_team_愛媛ＦＣ,away_team_東京ヴェルディ,away_team_松本山雅ＦＣ,away_team_柏レイソル,away_team_栃木ＳＣ,away_team_横浜Ｆ・マリノス,away_team_横浜ＦＣ,away_team_水戸ホーリーホック,away_team_浦和レッズ,away_team_清水エスパルス,away_team_湘南ベルマーレ,away_team_鹿島アントラーズ,away_team_ＦＣ岐阜,away_team_ＦＣ東京,away_team_ＦＣ町田ゼルビア,away_team_Ｖ・ファーレン長崎
0,13994,１,19694,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,13995,１,40000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
2,13996,１,21000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,13997,１,50000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,13998,１,39232,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,True,False,False,False,False,Fa

In [121]:
# datasにおいてtrainフラグが1のものをX_trainとする。その後、trainフラグを削除
X_train = datas[datas["train"] == 1]
X_train = X_train.drop(columns="train")

# datasにおいてtrainフラグが0のものをX_testとする。その後、trainフラグを削除
X_test = datas[datas["train"] == 0]
X_test = X_test.drop(columns="train")



# モデルごと


## LinearRegression


### 学習と予測


In [122]:
LRmodel = LR()
LRmodel.fit(X_train, y_train)
pred_LR = LRmodel.predict(X_test)

pred_LR[pred_LR<min_values] = min_values

### 予測値の保存


In [123]:
sample[1] = pred_LR
sample.to_csv("LR.csv", index=None, header=None)


## Ridge


In [124]:
ridge001 = Ridge(alpha=0.01)
ridge001.fit(X_train, y_train)
pred_ridge001 = ridge001.predict(X_test)
pred_ridge001[pred_ridge001<min_values] = min_values
sample[1] = pred_ridge001
sample.to_csv("Ridge001.csv", index=None, header=None)


In [125]:
ridge01 = Ridge(alpha=0.1)
ridge01.fit(X_train, y_train)
pred_ridge01 = ridge01.predict(X_test)
pred_ridge01[pred_ridge01<min_values] = min_values
sample[1] = pred_ridge01
sample.to_csv("Ridge01.csv", index=None, header=None)


In [126]:
ridge1 = Ridge()
ridge1.fit(X_train, y_train)
pred_ridge = ridge1.predict(X_test)
pred_ridge[pred_ridge<min_values] = min_values
sample[1] = pred_ridge
sample.to_csv("Ridge1.csv", index=None, header=None)


In [127]:
ridge10 = Ridge(alpha=10)
ridge10.fit(X_train, y_train)
pred_ridge10 = ridge10.predict(X_test)
sample[1] = pred_ridge10
sample.to_csv("Ridge10.csv", index=None, header=None)


## Lasso

In [128]:
lasso001 = Lasso(alpha=0.01, max_iter=100000)
lasso001.fit(X_train, y_train)
pred_lasso001 = lasso001.predict(X_test)
sample[1] = pred_lasso001
sample.to_csv("Lasso001.csv", index=None, header=None)

In [129]:
lasso1 = Lasso()
lasso1.fit(X_train, y_train)
pred_lasso1 = lasso1.predict(X_test)
sample[1] = pred_lasso1
sample.to_csv("Lasso1.csv", index=None, header=None)